# GCE Inputs for Enzo

This notebook provides the steps to generate the input table for Enzo to include chemical species and radiation in the feedback prescription of the stellar particles.

Developers: Benoit Côté, Brian O'Shea

In [ ]:
# Standard Python package(s)
import numpy as np

# Stellar Yields for Galactic Modeling Applications (SYGMA)
# Ritter et al. (2018), http://adsabs.harvard.edu/abs/2018ApJS..237...42R
import sygma

**Note** Some of the scripts will be transfered in a background script away from the view of the users. This notebook is still under development.

---
## Stellar Population Parameters Selection

In [ ]:
# Initial mass function (IMF)
# "kroupa" --> Kroupa (2001)
# "chabrier" --> Chabrier (2003)
# "salpeter" --> Salpeter (1955)
# Possible to couple Chabrier + different index at high mass
# I will update this later on..
imf_type = 'kroupa'

# IMF full mass boundaries  [Msun]
imf_bdys = [0.1, 100.]

# IMF mass boundaries where yields are ejected [Msun]
imf_yields_range = [1., 100.]

# Number of Type Ia Supernovae (SNe Ia) per units of stellar mass formed
# Typically ~ 1e-3
nb_1a_per_m = 1.0e-3

# Stellar yield tables
yields_low_and_massive_stars = 'yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12delay.txt'
yields_SNe_Ia = 'yield_tables/sn1a_i99_W7.txt'

# Will be more options soon
# ..

# Create the parameter lists for SYGMA
kwargs = {"imf_type":imf_type, "imf_bdys":imf_bdys, "imf_yields_range":imf_yields_range,\
          "nb_1a_per_m":nb_1a_per_m, "table":yields_low_and_massive_stars,\
          "sn1a_table":yields_SNe_Ia}

#### Metallicities and Elements  Selection

In [ ]:
# List of metallicities to appear on the input Enzo table 
# Z is the mass fraction of metals, NOT normalized to Solar
Z_list = [0.02, 0.01, 0.001]
len_Z_list = len(Z_list)

# List of elements
elem_list = ['H', 'O', 'C', 'N', 'Mg', 'S', 'Si', 'Ne', 'Fe', 'Mn', 'Ba', 'Eu', 'Sr', 'La']
len_elem_list = len(elem_list)

#### Timestepping

In [ ]:
# See the following notebooks for more timesteps options
# https://github.com/NuGrid/NuPyCEE/blob/master/DOC/Capabilities/Timesteps_size_management.ipynb

# Logarithmic timesteps (longer timesteps at later times)
# - - - - - - - - - - - - - - - - - - - - - - - - - - - -
tend = 1.3e10 # Final time of the SYGMA simulations [yr]
nb_dt = 30   # Total number of timesteps

# Update list of parameters for SYGMA
kwargs["tend"] = tend
kwargs["special_timesteps"] = nb_dt

## Get Outputs from SYGMA

In [ ]:
# Run SYGMA for all metallicities
sygma_list = []
for Z in Z_list:
    sygma_list.append(sygma.sygma(iniZ=Z, **kwargs))
    
    # Check whether the elements are available
    # This will go in a background script
    not_available = []
    for elem in elem_list:
        if not elem in sygma_list[-1].history.elements:
            not_available.append(elem)
    if len(not_available) > 0:
        for elem in not_available:
            print(elem,' not available with the selected yields tables.')
        print('SYGMA run stopped.')
        break

In [ ]:
# Get the list of isotope indexes for each selected elements
# This will go in a background script
i_iso = {}
for elem in elem_list:
    i_iso[elem] = []
for i in range(sygma_list[0].nb_isotopes):
    elem = sygma_list[0].history.isotopes[i].split('-')[0]
    if elem in elem_list:
        i_iso[elem].append(i)

In [ ]:
# Get the mass ejection rate [Msun/yr] of each element at each timestep for each metallicity
# m_ej_X_rate[ metallicity index ][ timestep index ][ element index ] --> Element mass
m_ej_X_rate = np.zeros((len_Z_list, nb_dt, len_elem_list))

# This will go in a background script
# For each metallicity ..
for i_Z in range(len_Z_list):
    
    # For each timestep ..
    for i_t in range(nb_dt):
        
        # For each element ..
        for i_e in range(len_elem_list):
            
            # For each isotope of that element ..
            for i_i in i_iso[elem_list[i_e]]:
                
                # Accumulate the total ejected mass for that isotope, at that timestep
                m_ej_X_rate[i_Z][i_t][i_e] += sygma_list[i_Z].mdot[i_t][i_i]
        
        # Convert total mass [Msun] into rate [Msun/yr]
        m_ej_X_rate[i_Z][i_t] /= sygma_list[i_Z].history.timesteps[i_t]

# Get the total and metal mass ejection rate [Msun/yr] at each timestep for each metallicity
# m_ej_Z_rate[ metallicity index ][ timestep index ] --> Metal mass
# m_ej_rate[ metallicity index ][ timestep index ] --> Total mass
m_ej_rate = np.zeros((len_Z_list, nb_dt))
m_ej_Z_rate = np.zeros((len_Z_list, nb_dt))

# This will go in a background script
# For each metallicity ..
non_Z_elem = ['H', 'He', 'Li']
for i_Z in range(len_Z_list):
    
    # For each timestep ..
    for i_t in range(nb_dt):
        
        # For each isotope ..
        for i_i in range(sygma_list[i_Z].nb_isotopes):
            
            # Add metals and total mass ejected
            m_ej_rate[i_Z][i_t] += sygma_list[i_Z].mdot[i_t][i_i]
            if not sygma_list[i_Z].history.isotopes[i_i].split('-')[0] in non_Z_elem:
                m_ej_Z_rate[i_Z][i_t] += sygma_list[i_Z].mdot[i_t][i_i]
            
    # Convert total mass [Msun] into rate [Msun/yr]
    m_ej_rate[i_Z][i_t] /= sygma_list[i_Z].history.timesteps[i_t]
    m_ej_Z_rate[i_Z][i_t] /= sygma_list[i_Z].history.timesteps[i_t]

In [ ]:
# Get the time values at the center of each timestep (i.e., each time bin)
# This will go in a background script
t_center_bin = np.zeros(nb_dt)
for i_t in range(nb_dt):
    t_center_bin[i_t] = sygma_list[0].history.age[i_t] + \
        0.5*sygma_list[0].history.timesteps[i_t]

In [ ]:
# Include dummy radiation values ..
dE_dt_mec = np.zeros((len_Z_list, nb_dt))
dE_dt_lum_bol = np.zeros((len_Z_list, nb_dt))
dE_dt_lum_ionizing = np.zeros((len_Z_list, nb_dt))
dE_dt_lum_dissociating = np.zeros((len_Z_list, nb_dt))
dE_dt_lum_electric = np.zeros((len_Z_list, nb_dt))

## Create Enzo Input Table

In [ ]:
comment_symbol = '# '

In [ ]:
# Write the header with information about columns and stellar population parameters
def write_header(text_file):
    
    # Initialize the header
    the_header = ''
    
    # Write general information
    the_header += comment_symbol+'Input table generated using SYGMA (Ritter et al. 2018)\n'
    
    # Write the stellar population parameters
    the_header += comment_symbol+'\n'
    the_header += comment_symbol+'Stellar population parameters\n'
    the_header += comment_symbol+'=============================\n'
    for key in kwargs.keys():
        the_header += comment_symbol+key+' = '+str(kwargs[key])+'\n'
        
    # Write columns information
    the_header += comment_symbol+'\n'
    the_header += comment_symbol+'Columns\n'
    the_header += comment_symbol+'=======\n'
    the_header += comment_symbol+'1.  Time    [yr] (middle of time bin)\n'
    the_header += comment_symbol+'2.  dM/dt   [Msun/yr] (total)\n'
    the_header += comment_symbol+'3.  dM_Z/dt [Msun/yr] (metals)\n'
    for i_e in range(len_elem_list):
        if (4+i_e) > 9:
            sp = ' '
        else:
            sp = '  '
        the_header += comment_symbol+str(4+i_e)+'.'+sp+'dM_X/dt [Msun/yr] ('+elem_list[i_e]+')\n'
    i_col = 4+i_e
    the_header += comment_symbol+str(i_col+1)+'. dE/dt   [??] (mechanical)\n'
    the_header += comment_symbol+str(i_col+2)+'. dE/dt   [??] (bolometric)\n'
    the_header += comment_symbol+str(i_col+3)+'. dE/dt   [??] (photoionizing, 13.6 eV and up)\n'
    the_header += comment_symbol+str(i_col+4)+'. dE/dt   [??] (photodissociating, LW, 11.18 - 13.6 eV)\n'
    the_header += comment_symbol+str(i_col+5)+'. dE/dt   [??] (photoelectric heating, 8 - 13.6 eV)\n'
    
    # Add header
    text_file.write(the_header)

In [ ]:
# Define the format of the digits in the table
form = '%.4E'
extra_space = '  '

# Open the output file
text_file = open("/Users/benoitcote/Desktop/enzo_gce_table.txt", "w")
write_header(text_file)

# For each metallicity ..
for i_Z in range(len_Z_list):
    
    # Write the metallicity header
    Z_header = ''
    Z_header += comment_symbol+'\n'
    Z_header += comment_symbol+'Z = '+str(Z_list[i_Z])+'\n'
    Z_header += comment_symbol+'==========\n'
    text_file.write(Z_header)

    # Declare the block of data for that metallicity
    the_block = ''

    # For each time bin ..
    for i_t in range(nb_dt):

        # Create an empty line
        the_line = ''

        # Add the time (center of the time bin) [yr]
        the_line += str(form%t_center_bin[i_t]) + extra_space

        # Add the total and metal mass ejected rate [Msun/yr]
        the_line += str(form%m_ej_rate[i_Z][i_t]) + extra_space
        the_line += str(form%m_ej_Z_rate[i_Z][i_t]) + extra_space

        # Add the elemental mass ejected rate [Msun/yr]
        for i_e in range(len_elem_list):
            the_line += str(form%m_ej_X_rate[i_Z][i_t][i_e]) + extra_space

        # Add luminosities
        the_line += str(form%dE_dt_mec[i_Z][i_t]) + extra_space
        the_line += str(form%dE_dt_lum_bol[i_Z][i_t]) + extra_space
        the_line += str(form%dE_dt_lum_ionizing[i_Z][i_t]) + extra_space
        the_line += str(form%dE_dt_lum_dissociating[i_Z][i_t]) + extra_space
        the_line += str(form%dE_dt_lum_electric[i_Z][i_t])

        # Add the line to the block
        the_block += the_line + '\n'

    # Write the block in the output file
    text_file.write(the_block)
    
# Close the output file
text_file.close()